In [1]:
import numpy as np
import pickle
import pathlib

from os.path import join, exists
path_to_file = str(pathlib.Path().resolve())
dir_path = join(path_to_file, "../../")

import sys
sys.path.append(join(dir_path, "HelperFiles"))

from helper import *
np.set_printoptions(suppress=True)


# SHAP Results

In [2]:
datasets = ["census", "bank", "brca", "credit", "breast_cancer"]
Ks = [2,5]
methods = ["rankshap", "sprtshap"]
guarantees = ["rank", "set"]
for method in methods:
    for K in Ks:
        for alpha in [0.1, 0.2]:
            max_mat = np.empty((len(Ks), len(datasets)))
            for i, guarantee in enumerate(guarantees):
                data_dir = join(dir_path, "Experiments", "Results", "Top_K", guarantee, "alpha_"+str(alpha)) 
                for j, dataset in enumerate(datasets):
                    max_mat[i,j] = np.nan
                    fname = method + "_" + dataset + "_K" + str(K) 
                    path = join(data_dir, fname)
                    if exists(path):
                        with open(path, "rb") as fp:
                            results = pickle.load(fp)
                        max_fwer = calc_max_fwer(results)
                        if max_fwer is not None:
                            max_mat[i, j] = np.round(max_fwer*100)
            print("#"*20)
            print("Method: ", method, "\tK:", str(K), "\tAlpha:", alpha)

            print("Max FWER (%)")
            print(max_mat)


####################
Method:  rankshap 	K: 2 	Alpha: 0.1
Max FWER (%)
[[8. 6. 6. 4. 0.]
 [2. 2. 6. 2. 4.]]
####################
Method:  rankshap 	K: 2 	Alpha: 0.2
Max FWER (%)
[[16. 14. 14.  8. 10.]
 [16.  0. 10.  2. 12.]]
####################
Method:  rankshap 	K: 5 	Alpha: 0.1
Max FWER (%)
[[ 6. 10. 10.  4.  6.]
 [ 6. 10. 10.  4.  4.]]
####################
Method:  rankshap 	K: 5 	Alpha: 0.2
Max FWER (%)
[[14. 18. 20. 12. 20.]
 [14. 16. 20. 16.  4.]]
####################
Method:  sprtshap 	K: 2 	Alpha: 0.1
Max FWER (%)
[[ 0.  0. nan  0.  0.]
 [ 0.  0.  0.  0.  0.]]
####################
Method:  sprtshap 	K: 2 	Alpha: 0.2
Max FWER (%)
[[ 2.  2. nan  0.  4.]
 [ 0.  2.  0.  0.  0.]]
####################
Method:  sprtshap 	K: 5 	Alpha: 0.1
Max FWER (%)
[[nan nan nan nan nan]
 [ 0.  0.  0.  0.  0.]]
####################
Method:  sprtshap 	K: 5 	Alpha: 0.2
Max FWER (%)
[[nan nan nan nan  0.]
 [ 8.  2.  2.  2.  0.]]


## LIME Results

In [3]:
method = "lime"
guarantee = "rank"
alphas = [0.1, 0.2]
for K in Ks:
    max_mat = np.full((len(alphas), len(datasets)), np.nan)
    for i, alpha in enumerate(alphas):
        data_dir = join(dir_path, "Experiments", "Results", "Top_K", guarantee, "alpha_"+str(alpha))
        for j, dataset in enumerate(datasets):
            fname = method + "_" + dataset + "_K" + str(K) 
            path = join(data_dir, fname)
            if exists(path):
                with open(path, "rb") as fp:
                    results = pickle.load(fp)
                max_fwer = calc_max_fwer(results)
                if max_fwer is not None:
                    max_mat[i, j] = np.round(max_fwer*100)

    print("#"*20)
    print("Method: LIME", "\tK: ", K)

    print("Max FWER (%)")
    print(max_mat)



####################
Method: LIME 	K:  2
Max FWER (%)
[[0. 2. 2. 6. 2.]
 [2. 0. 0. 8. 2.]]
####################
Method: LIME 	K:  5
Max FWER (%)
[[nan  0. nan  0.  0.]
 [nan  0. nan  0.  0.]]
